<a href="https://colab.research.google.com/github/sruthi1996/AlternusVera/blob/master/Alternusvera_sprint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Topic: AlternaVera**
Datasets Used: 
* **Liar Liar Dataset**: Dataset contains news statements classified as a range of values from True to PantsOnFire; having 6 classes. Classifies the statement to be either true or barely true or mostly true ansd so on.
* **Fake News dataset** from the Kaggle which classifies based on Author and Text in the document.

**Factor: Content Statistics**

For any text based applications; it is important to understand the statistics or metrics of any content/data that is undergoing processing.

Applications like textual analysis,  information retrival or text summerization use these metrics to better understand the term's frequency's , their importance in the given document.

While the term document frequency refers to the bag of words, The inverse document frequency is the value of how many times a word occurs in a document.
Here are the steps to be followed:
* Preprocess or clean the document
* Creating a count vector of all the terms
* Building a tf-idf matrix
* scoring or ranking each sentence in the dataset

In [0]:
import pandas as pd
import numpy as np
import csv
import re
import seaborn as sns
from google.colab import drive


In [0]:
drive.mount('/content/drive')
fake_train = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/fakenews/train_fakenews.csv",sep=',')
fake_test = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/fakenews/test_fakenews.csv",sep=',')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
print(fake_train.shape)
print(fake_test.shape)

(20800, 5)
(5200, 4)


In [0]:
fake_train.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [0]:
fake_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [0]:
fake_train['speaker']=fake_train['author']

In [0]:
fake_train.drop(['author','id'], axis = 1)

,title,text,label,speaker
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,Daniel J. Flynn
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,0,Benjamin Hoffman
20797,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams
20798,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",1,Alex Ansary


In [0]:
fake_train['speaker']

0                                    Darrell Lucus
1                                  Daniel J. Flynn
2                               Consortiumnews.com
3                                  Jessica Purkiss
4                                   Howard Portnoy
                           ...                    
20795                                Jerome Hudson
20796                             Benjamin Hoffman
20797    Michael J. de la Merced and Rachel Abrams
20798                                  Alex Ansary
20799                                David Swanson
Name: speaker, Length: 20800, dtype: object

In [0]:
fake_train['label']

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [0]:
liar_train = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/liar_dataset/train.tsv",sep='\t')
liar_test = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/liar_dataset/test.tsv",sep='\t')
liar_valid = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/liar_dataset/valid.tsv",sep='\t')


In [0]:
print(liar_train.shape)
print(liar_test.shape)
print(liar_valid.shape)

(10239, 14)
(1266, 14)
(1283, 14)


In [0]:
columns = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

In [0]:
liar_train.columns = columns
liar_test.columns = columns
liar_valid.columns = columns

In [0]:
liar_train.head(3)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release


Now that we have loaded both the datasets; it's clear that the Number of classes/labels present in the Fakenews and Liar dataset are different.

The Classes/labels present in Fake News dataset are: 0 and 1

The classes/labels present in the Liar dataset are: "False","True","barely-true", "half-true", "mostly-true", "pants-fire".

For a better classification; we combine the labels in the following way:

* "False", "half-true", "barely-true", "pants-fire" as lies i.e. class 0
* "True" & "mostly-true" as truth i.e. class 1



In [0]:
liar_train_new=liar_train
#liar_train_new['new_label'] = liar_train_new['label'].map({False: 0, True: 1, 'barely-true': 0, 'half-true': 0,'mostly-true': 1,'pants-fire': 0 })


In [0]:
cat_to_nums = {"label":     {"true":1, "false":0, "barely-true": 0, "half-true": 0, "mostly-true": 1,"pants-fire": 0} }
liar_train_new.replace(cat_to_nums, inplace=True)
liar_train_new.head()


,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [0]:
liar_train_new["label"]

0        0
1        1
2        0
3        0
4        1
        ..
10234    1
10235    1
10236    0
10237    0
10238    0
Name: label, Length: 10239, dtype: int64

In [0]:
liar_train_new.isna()

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10234,False,False,False,False,False,True,False,False,False,False,False,False,False,False
10235,False,False,False,False,False,True,False,False,False,False,False,False,False,False
10236,False,False,False,False,False,True,False,False,False,False,False,False,False,False
10237,False,False,False,False,False,True,False,False,False,False,False,False,False,False


As, we can see there are few rows which returned True above which means the rows have missing data. Lets remove the rows with this missing data as they of negligible number.

In [0]:
liar_train_new = liar_train_new.dropna(axis=0)

In [0]:
liar_train_new.isna()

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10226,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10227,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10229,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10230,False,False,False,False,False,False,False,False,False,False,False,False,False,False


**Data amalgamation :** merging the Fake news dataset with the liar dataset based on the "new_label" column;
To do this; we have to rename the "label" feature present in  the fake news dataset as "new_label".

In [0]:
#fake_train.rename(columns = {'label':'new_label'}, inplace = True)

In [0]:
merged_data = pd.merge(liar_train_new, fake_train, on="speaker", how='left')

In [0]:
merged_data.head(5)

,jsonid,label_x,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,id,title,author,text,label_y
0,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,NaN,NaN,NaN,NaN,NaN
1,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,NaN,NaN,NaN,NaN,NaN
2,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,NaN,NaN,NaN,NaN,NaN
3,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa.",NaN,NaN,NaN,NaN,NaN
4,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video,NaN,NaN,NaN,NaN,NaN


In [0]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6723 entries, 0 to 6722
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   jsonid             6723 non-null   object 
 1   label_x            6723 non-null   int64  
 2   headline_text      6723 non-null   object 
 3   subject            6723 non-null   object 
 4   speaker            6723 non-null   object 
 5   speakerjobtitle    6723 non-null   object 
 6   stateinfo          6723 non-null   object 
 7   partyaffiliation   6723 non-null   object 
 8   barelytruecounts   6723 non-null   float64
 9   falsecounts        6723 non-null   float64
 10  halftruecounts     6723 non-null   float64
 11  mostlytrueocunts   6723 non-null   float64
 12  pantsonfirecounts  6723 non-null   float64
 13  context            6723 non-null   object 
 14  id                 0 non-null      float64
 15  title              0 non-null      object 
 16  author             0 non

As we can see, we have datatypes of few features as either object or float. The float values have to be converted 

In [0]:
#merged_data = merged_data["new_label_x"].astype(int)


In [0]:
#merged_data = merged_data["new_label_y"].astype(int)

creating a checkpoint here and importing the merged data to a csv file

In [0]:
merged_data.to_csv("mergeddata.csv", index=False)

In [0]:
print(merged_data.shape)

(6723, 19)


In [0]:
merged_data.to_csv("/content/drive/Shared drives/SheCodes/Datasets/merged_dataset/mergeddata.csv", index=False)

In [0]:
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from scipy import sparse

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
import nltk
nltk.download('vader_lexicon')
import warnings
import nltk.sentiment
warnings.filterwarnings('ignore')



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
fake = fake_train.drop(["title","author","speaker"],axis=1)
fake.head()

,id,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,Ever get the feeling your life circles the rou...,0
2,2,"Why the Truth Might Get You Fired October 29, ...",1
3,3,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
labels=fake_train.label
labels.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [0]:
#fake_train=fake_train["text"]

In [0]:
x_train,x_test,y_train,y_test=train_test_split(fake_train['text'],labels,test_size=0.2)

In [0]:
x_train.head()

9993     WikiLeaks Emails Uncover John Podesta’s Sick &...
12029    WASHINGTON  —   The United States is poring ov...
1291     Страна: Саудовская Аравия Как отмечает в своей...
18861    Cranberries are a beautiful addition to any di...
14166    GETTYSBURG, Pa.  —   Donald J. Trump came to t...
Name: text, dtype: object

In [0]:
#df1=liar_train["headline_text"]

##Feature: Content Statistics:

The statistics of content/corpus are those that can summerize it; explain various factors that help in measuring the document's size, word count, similarity, etc.

Various statistics for a document can be: 

Count Vector

tf-idf matrix


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect = count_vect.fit(liar_train["headline_text"])
freq_term_matrix = count_vect.transform(liar_train["headline_text"])

In [0]:
print(freq_term_matrix)

  (0, 751)	1
  (0, 1531)	1
  (0, 1963)	1
  (0, 2474)	1
  (0, 3184)	1
  (0, 3433)	1
  (0, 4859)	1
  (0, 4909)	1
  (0, 5686)	1
  (0, 6051)	1
  (0, 7417)	1
  (0, 7671)	1
  (0, 7673)	1
  (0, 8553)	1
  (0, 10424)	1
  (0, 10425)	2
  (0, 10979)	1
  (0, 10987)	1
  (0, 11109)	1
  (0, 11137)	1
  (0, 11933)	2
  (1, 856)	1
  (1, 1568)	1
  (1, 1961)	1
  (1, 1988)	1
  :	:
  (10238, 799)	1
  (10238, 1158)	1
  (10238, 2548)	1
  (10238, 2567)	1
  (10238, 3308)	1
  (10238, 3988)	1
  (10238, 5266)	1
  (10238, 6326)	1
  (10238, 6602)	1
  (10238, 6852)	1
  (10238, 7671)	2
  (10238, 7823)	1
  (10238, 7827)	1
  (10238, 8995)	1
  (10238, 10593)	1
  (10238, 10659)	1
  (10238, 10917)	1
  (10238, 10987)	1
  (10238, 11003)	1
  (10238, 11012)	1
  (10238, 11109)	2
  (10238, 11621)	1
  (10238, 11659)	2
  (10238, 12150)	2
  (10238, 12157)	1


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm="12")
tfidf.fit(freq_term_matrix)

TfidfTransformer(norm='12', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [0]:
#doc_freq_term = count_vect.transform([liar_test["headline_text"]])
#doc_tfidf_matrix = tfidf.transform(doc_freq_term)

In [0]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stop_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'or', 'that', 's', 'theirs', 'of', 'more', 'not', "you're", 'some', 'being', 'in', 'while', 'hasn', 'off', 'after', 'once', 'have', 'up', 'again', 'mightn', 'the', 'because', 'can', 'this', 'an', 'to', 'there', "wasn't", "won't", 'no', "haven't", 'these', 'd', 'themselves', 'nor', 'her', 'out', 'any', 'am', 'below', 'further', 'those', 'whom', 'only', 'me', 'needn', 'where', "don't", 're', 'll', 'had', 'ma', 'as', 'a', 'be', 'shouldn', 'won', 'if', 'both', "needn't", 'from', "you'll", 'are', 'should', 'himself', 'itself', 'so', 'what', 'his', 'which', "aren't", 'mustn', 'other', 'why', 'too', 'at', "doesn't", 'weren', 'wasn', 'has', 'do', 'was', 'above', "that'll", 'she', 'by', 'its', 'most', 'them', "she's", 'haven', 'yours', 'with', 'between', 'having', 'your', 'their', 'shan', 'about', "didn't", "shouldn't", 'than', "couldn't", 'against', 'my', "you'd", 'will', 'myself', '